Code Modified from
[inswapper](https://github.com/haofanwang/inswapper)
&
[CodeFormer](https://github.com/sczhou/CodeFormer)

In [ ]:
#@title Download From public drive link
import gdown
import os
def download_file(file_id, output_file):
    base_path = os.path.dirname(output_file)
    if not os.path.exists(base_path):
      os.makedirs(base_path)
    url = f'https://drive.google.com/uc?id={file_id}'
    gdown.download(url, output_file, quiet=False)
drive_id = "1-0QI7jHuJ7hkjfsdhkjfhkasdfk"  # @param {type: "string"}
download_file(drive_id, '/content/RoopFaceSwap.zip')
zip_path='/content/RoopFaceSwap.zip'

#or

In [2]:
#@title Access it from your google Dive (Private)
from google.colab import drive
drive.mount('/content/gdrive')
from IPython.display import clear_output
clear_output()
if os.path.exists("/content/gdrive/MyDrive/RoopFaceSwap_Backup/RoopFaceSwap.zip"):
  zip_path="/content/gdrive/MyDrive/RoopFaceSwap_Backup/RoopFaceSwap.zip"
else:
  print("Can't find RoopFaceSwap.zip")

In [4]:
#@title Extract zip file
import zipfile
import os
def extract_zip(zip_file_path,extract_path):
    # Create the extraction directory if it doesn't exist
    os.makedirs(extract_path, exist_ok=True)

    # Open the ZIP file
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # Extract all contents to the specified directory
        zip_ref.extractall(extract_path)

    print(f"Contents of '{zip_file_path}' extracted to '{extract_path}'.")

extract_zip(zip_path,'/content/RoopFaceSwap')

Contents of '/content/gdrive/MyDrive/RoopFaceSwap_Backup/RoopFaceSwap.zip' extracted to '/content/RoopFaceSwap'.


In [ ]:
#@title Install and restart
!pip install -r /content/RoopFaceSwap/requirements.txt
from IPython.display import clear_output
clear_output()
import time
time.sleep(3)
import os
os.kill(os.getpid(), 9)

In [43]:
#@title utils
"""
This project is developed by Haofan Wang to support face swap in single frame. Multi-frame will be supported soon!

It is highly built on the top of insightface, sd-webui-roop and CodeFormer.
"""

import os
import cv2
import copy
import argparse
import insightface
import onnxruntime
import numpy as np
from PIL import Image
from typing import List, Union, Dict, Set, Tuple


# Set the CUDA_VISIBLE_DEVICES environment variable to the index of your GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # Replace '0' with the index of your GPU

# Rest of your script...

def getFaceSwapModel(model_path: str):
    model = insightface.model_zoo.get_model(model_path)
    return model


# def getFaceAnalyser(model_path: str, providers,
#                     det_size=(320, 320)):

#     face_analyser = insightface.app.FaceAnalysis(name="buffalo_l", root="/content/inswapper/checkpoints", providers=providers)
#     face_analyser.prepare(ctx_id=0, det_size=det_size)
#     return face_analyser

def getFaceAnalyser(model_path: str, providers, det_size=(320, 320)):
    print("Available Providers before:", onnxruntime.get_available_providers())
    face_analyser = insightface.app.FaceAnalysis(
        name="buffalo_l", root="/content/RoopFaceSwap/inswapper/checkpoints", providers=providers
    )
    print("Available Providers after:", onnxruntime.get_available_providers())
    face_analyser.prepare(ctx_id=0, det_size=det_size)
    return face_analyser


def get_one_face(face_analyser,
                 frame:np.ndarray):
    face = face_analyser.get(frame)
    try:
        return min(face, key=lambda x: x.bbox[0])
    except ValueError:
        return None


def get_many_faces(face_analyser,
                   frame:np.ndarray):
    """
    get faces from left to right by order
    """
    try:
        face = face_analyser.get(frame)
        return sorted(face, key=lambda x: x.bbox[0])
    except IndexError:
        return None


def swap_face(face_swapper,
              source_faces,
              target_faces,
              source_index,
              target_index,
              temp_frame):
    """
    paste source_face on target image
    """
    source_face = source_faces[source_index]
    target_face = target_faces[target_index]

    return face_swapper.get(temp_frame, target_face, source_face, paste_back=True)


def process(source_img: Union[Image.Image, List],
            target_img: Image.Image,
            source_indexes: str,
            target_indexes: str,
            model: str):
    # load machine default available providers
    # providers = onnxruntime.get_available_providers()
    # providers=['CUDAExecutionProvider']
    providers=['CUDA']
    # load face_analyser
    face_analyser = getFaceAnalyser(model, providers)

    # load face_swapper
    model_path = model
    face_swapper = getFaceSwapModel(model_path)

    # read target image
    target_img = cv2.cvtColor(np.array(target_img), cv2.COLOR_RGB2BGR)

    # detect faces that will be replaced in the target image
    target_faces = get_many_faces(face_analyser, target_img)
    num_target_faces = len(target_faces)
    num_source_images = len(source_img)

    if target_faces is not None:
        temp_frame = copy.deepcopy(target_img)
        if isinstance(source_img, list) and num_source_images == num_target_faces:
            print("Replacing faces in target image from the left to the right by order")
            for i in range(num_target_faces):
                source_faces = get_many_faces(face_analyser, cv2.cvtColor(np.array(source_img[i]), cv2.COLOR_RGB2BGR))
                source_index = i
                target_index = i

                if source_faces is None:
                    raise Exception("No source faces found!")

                temp_frame = swap_face(
                    face_swapper,
                    source_faces,
                    target_faces,
                    source_index,
                    target_index,
                    temp_frame
                )
        elif num_source_images == 1:
            # detect source faces that will be replaced into the target image
            source_faces = get_many_faces(face_analyser, cv2.cvtColor(np.array(source_img[0]), cv2.COLOR_RGB2BGR))
            num_source_faces = len(source_faces)
            print(f"Source faces: {num_source_faces}")
            print(f"Target faces: {num_target_faces}")

            if source_faces is None:
                raise Exception("No source faces found!")

            if target_indexes == "-1":
                if num_source_faces == 1:
                    print("Replacing all faces in target image with the same face from the source image")
                    num_iterations = num_target_faces
                elif num_source_faces < num_target_faces:
                    print("There are less faces in the source image than the target image, replacing as many as we can")
                    num_iterations = num_source_faces
                elif num_target_faces < num_source_faces:
                    print("There are less faces in the target image than the source image, replacing as many as we can")
                    num_iterations = num_target_faces
                else:
                    print("Replacing all faces in the target image with the faces from the source image")
                    num_iterations = num_target_faces

                for i in range(num_iterations):
                    source_index = 0 if num_source_faces == 1 else i
                    target_index = i

                    temp_frame = swap_face(
                        face_swapper,
                        source_faces,
                        target_faces,
                        source_index,
                        target_index,
                        temp_frame
                    )
            else:
                print("Replacing specific face(s) in the target image with specific face(s) from the source image")

                if source_indexes == "-1":
                    source_indexes = ','.join(map(lambda x: str(x), range(num_source_faces)))

                if target_indexes == "-1":
                    target_indexes = ','.join(map(lambda x: str(x), range(num_target_faces)))

                source_indexes = source_indexes.split(',')
                target_indexes = target_indexes.split(',')
                num_source_faces_to_swap = len(source_indexes)
                num_target_faces_to_swap = len(target_indexes)

                if num_source_faces_to_swap > num_source_faces:
                    raise Exception("Number of source indexes is greater than the number of faces in the source image")

                if num_target_faces_to_swap > num_target_faces:
                    raise Exception("Number of target indexes is greater than the number of faces in the target image")

                if num_source_faces_to_swap > num_target_faces_to_swap:
                    num_iterations = num_source_faces_to_swap
                else:
                    num_iterations = num_target_faces_to_swap

                if num_source_faces_to_swap == num_target_faces_to_swap:
                    for index in range(num_iterations):
                        source_index = int(source_indexes[index])
                        target_index = int(target_indexes[index])

                        if source_index > num_source_faces-1:
                            raise ValueError(f"Source index {source_index} is higher than the number of faces in the source image")

                        if target_index > num_target_faces-1:
                            raise ValueError(f"Target index {target_index} is higher than the number of faces in the target image")

                        temp_frame = swap_face(
                            face_swapper,
                            source_faces,
                            target_faces,
                            source_index,
                            target_index,
                            temp_frame
                        )
        else:
            raise Exception("Unsupported face configuration")
        result = temp_frame
    else:
        print("No target faces found!")

    result_image = Image.fromarray(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
    return result_image


import matplotlib.pyplot as plt
import matplotlib.image as mpimg


import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def display_images(donor_image_path, target_image_path, swapped_face_path, upscale_swap_face_path):
    # Load images
    donor_image = mpimg.imread(donor_image_path)
    target_image = mpimg.imread(target_image_path)
    swapped_face = mpimg.imread(swapped_face_path)
    upscale_swap_face = mpimg.imread(upscale_swap_face_path)

    # Create a figure with 1 row and 4 columns
    fig, axs = plt.subplots(1, 4, figsize=(16, 4))

    # Display donor image
    axs[0].imshow(donor_image)
    axs[0].axis('off')
    axs[0].set_title('Source Image')

    # Display target image
    axs[1].imshow(target_image)
    axs[1].axis('off')
    axs[1].set_title('Target Image')

    # Display swapped face
    axs[2].imshow(swapped_face)
    axs[2].axis('off')
    axs[2].set_title('Swapped Face')

    # Display upscaled swapped face
    axs[3].imshow(upscale_swap_face)
    axs[3].axis('off')
    axs[3].set_title('Upscaled Swapped Face')

    # Adjust layout for better spacing
    plt.tight_layout()

    # Show the plot
    plt.show()

# Example usage:
# display_images("donor_image.jpg", "target_image.jpg", "swapped_face.jpg", "upscale_swap_face.jpg")

def display_image(image_path):
    # Read the image using matplotlib.image.imread
    img = mpimg.imread(image_path)

    # Display the image using matplotlib.pyplot.imshow
    plt.imshow(img)
    plt.axis('off')  # Turn off axis labels
    plt.show()









In [37]:
#@title Upload Doner Image (Whose face you want to copy)
import os
from google.colab import files
import shutil

upload_folder = '/content/user_upload'

if not os.path.exists(upload_folder):
    os.mkdir(upload_folder)
target_person=[]
uploaded = files.upload()
for filename in uploaded.keys():
  dst_path = os.path.join(upload_folder, filename)
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)
  target_person.append(dst_path)

from IPython.display import clear_output
clear_output()
target_person[-1]

'/content/user_upload/SnoopDogg.jpg'

In [ ]:
Source_Image = "/content/user_upload/SnoopDogg.jpg"  # @param {type: "string"}
display_image(Source_Image)



In [39]:

#@title Upload Target Image (Where you want to paste the face)
import os
from google.colab import files
import shutil

upload_folder = '/content/user_upload'

if not os.path.exists(upload_folder):
    os.mkdir(upload_folder)

doner=[]
uploaded = files.upload()
for filename in uploaded.keys():
  dst_path = os.path.join(upload_folder, filename)
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)
  doner.append(dst_path)

from IPython.display import clear_output
clear_output()
doner[-1]

'/content/user_upload/jack.jpg'

In [ ]:
Target_Image = "/content/user_upload/jack.jpg"  # @param {type: "string"}
display_image(Target_Image)

In [ ]:
import os
import shutil
import uuid

def upscale(upload_image,BACKGROUND_ENHANCE, FACE_UPSAMPLE, CODEFORMER_FIDELITY):
    if os.getcwd() != "/content/RoopFaceSwap/CodeFormer":
        os.chdir("/content/RoopFaceSwap/CodeFormer")
    save_folder = "/content/Swapped_face"
    if not os.path.exists(save_folder):
        os.mkdir(save_folder)



    file_name_with_extension = os.path.basename(upload_image)
    file_name_without_extension, file_extension = os.path.splitext(file_name_with_extension)
    random_uuid = str(uuid.uuid4())
    random_name = random_uuid[:8]

    if os.path.exists("/content/temp_image"):
        shutil.rmtree("/content/temp_image")
    os.mkdir("/content/temp_image")
    shutil.copy(upload_image, f"/content/temp_image/{random_name}{file_extension}")

    delete_path = f"/content/RoopFaceSwap/CodeFormer/results/temp_image_{CODEFORMER_FIDELITY}"
    if os.path.exists(delete_path):
        shutil.rmtree(delete_path)

    save_path = f"{delete_path}/final_results/{random_name}.png"
    copy_path = f"/content/Swapped_face/swap_{file_name_without_extension}.png"

    try:
        if BACKGROUND_ENHANCE:
            if FACE_UPSAMPLE:
                command=f"python inference_codeformer.py -w {CODEFORMER_FIDELITY} --input_path /content/temp_image --bg_upsampler realesrgan --face_upsample"
                var1 = os.system(command)
                print(f"var1 output: {var1}")
                if var1 == 0:
                    shutil.copy(save_path, copy_path)
                    return copy_path
                else:
                    print(command)
                    print("Failed to Upscale")
            else:
                command=f"python inference_codeformer.py -w {CODEFORMER_FIDELITY} --input_path /content/temp_image --bg_upsampler realesrgan"
                var2 = os.system(command)
                print(f"var2 output: {var2}")
                if var2 == 0:
                    shutil.copy(save_path, copy_path)
                    return copy_path
                else:
                    print(command)
                    print("Failed to Upscale")
        else:
            command=f"python inference_codeformer.py -w {CODEFORMER_FIDELITY} --input_path /content/temp_image"
            var3 = os.system(command)
            print(f"var3 output: {var3}")
            if var3 == 0:
                shutil.copy(save_path, copy_path)
                return copy_path
            else:
                print("Failed to Upscale")
                print(command)
    except Exception as e:
        print(f"An exception occurred: {e}")

    return None


import os
from IPython.display import clear_output

def roop_face_swap(Target_Image,Source_Image, BACKGROUND_ENHANCE, FACE_UPSAMPLE, CODEFORMER_FIDELITY):
  source_img = [Image.open(Source_Image)]
  target_img = Image.open(Target_Image)
  model = "/content/RoopFaceSwap/inswapper/inswapper_128.onnx"
  result_image = process(source_img, target_img, -1, -1, model)
  if not os.path.exists("/content/low_quality"):
    os.mkdir("/content/low_quality")
  base_filename = os.path.basename(Target_Image)
  result_image.save(f"/content/low_quality/low_{base_filename}")
  final_save_path=upscale(f"/content/low_quality/low_{base_filename}", BACKGROUND_ENHANCE, FACE_UPSAMPLE, CODEFORMER_FIDELITY)
  clear_output()
  return f"/content/low_quality/low_{base_filename}",final_save_path




  # Inference the uploaded images
#@markdown `CODEFORMER_FIDELITY`: Balance the quality (lower number) and fidelity (higher number)<br>
# you can add '--bg_upsampler realesrgan' to enhance the background
CODEFORMER_FIDELITY = 0.7 #@param {type:"slider", min:0, max:1, step:0.01}
#@markdown `BACKGROUND_ENHANCE`: Enhance background image with Real-ESRGAN<br>
BACKGROUND_ENHANCE = True #@param {type:"boolean"}
#@markdown `FACE_UPSAMPLE`: Upsample restored faces for high-resolution AI-created images<br>
FACE_UPSAMPLE = False #@param {type:"boolean"}
swap_face_path,upscale_swap_face_path=roop_face_swap(Target_Image,Source_Image, BACKGROUND_ENHANCE, FACE_UPSAMPLE, CODEFORMER_FIDELITY)
display_images(Source_Image, Target_Image, swap_face_path,upscale_swap_face_path)


In [ ]:
#@title Download Swapped Face
display_image(swap_face_path)
from google.colab import files
files.download(swap_face_path)

In [ ]:
#@title Download Upscale Swapped Face
display_image(upscale_swap_face_path)
from google.colab import files
files.download(upscale_swap_face_path)

In [ ]:
#@title Gradio Link try yourself

# # !pip install gradio==3.47.1
# import gradio as gr

# import numpy as np
# from PIL import Image
# def save_pil_to_normal_image(pil_image, path):
#     # Save the PIL image to a normal image
#     pil_image.save(path)

# def roop_face_swap(target_image, source_image, background_enhance, face_upsample, rescaling_factor, codeformer_fidelity):
#     # Implement your face swap logic here
#     # Make sure to return the output image as a numpy array
#     # Example: output_image = face_swap_function(target_image, source_image, background_enhance, face_upsample, rescaling_factor, codeformer_fidelity)
#     output_image = np.zeros((256, 256, 3))  # Replace this with your actual output

#     return output_image

# def face_swap_interface(target_image, source_image, background_enhance, face_upsample, rescaling_factor, codeformer_fidelity):
#     # Convert Gradio Image objects to PIL Image objects
#     target_pil = Image.fromarray(np.uint8(target_image))
#     source_pil = Image.fromarray(np.uint8(source_image))

#     # Save PIL images to normal images
#     save_pil_to_normal_image(target_pil, "/content/target_image.jpg")
#     save_pil_to_normal_image(source_pil, "/content/source_image.jpg")

#     # Call your face swap function
#     output_image = roop_face_swap("/content/target_image.jpg", "/content/source_image.jpg", background_enhance, face_upsample, codeformer_fidelity)

#     return output_image

# # Create Gradio Interface
# demo = gr.Interface(
#     fn=face_swap_interface,
#     inputs=[
#         gr.Image(shape=None, type="numpy", label='Target'),
#         gr.Image(shape=None, type="numpy", label='Source'),
#         gr.Checkbox(value=True, label="Background_Enhance"),
#         gr.Checkbox(value=True, label="Face_Upsample"),
#         gr.Slider(0, 1, value=0.5, step=0.01, label='Codeformer_Fidelity (0 for better quality, 1 for better identity)')
#     ],
#     outputs=gr.Image(type="numpy", label="Output").style(height='auto'),
#     title="Face swap"
# )

# # Launch Gradio Interface
# demo.launch()
